# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

zrobic rozszerzony algorytm euklidesa w wersji rekurencyjnej

In [1]:
## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera.

test aks mozna zrobic

In [4]:
import random
def isPrime3(n, k=10):
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False    # speedup

    s = 0
    d = n-1
    while d % 2 == 0:
        s += 1
        d //= 2

    for i in range(k):
        a = random.randrange(2, n-2)    # 2 <= a <= n-2
        # x = (a**d) % n
        x = pow(a, d, n)
        if (x != 1) and (x!= (n-1)):  #x^d przystaje 1 mod n (warunek silne pseudopierwszości) testujemy nowego swiadka
            for j in range(1,s):
                if (x== n-1): break 
                x = (x**2) % n
            if x != n-1: return False
    return True    

for i in range(65500,65600):            
    if(isPrime3(i,10)): print(i)

65519
65521
65537
65539
65543
65551
65557
65563
65579
65581
65587
65599


## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [12]:
#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize):
    #napisz swój kod tutaj
    num = random.randrange(2**(keysize-1), 2**keysize)

    while not isPrime3(num):
        num = random.randrange(2**(keysize-1), 2**keysize)

    return num

generatePrime(2048)

30193501776225272622461490244550108320644841587936971897204016572189868048469293945298593143053430214674928072845610393913629679502970038990683551794144913307073885459328957599767354494644915506723892945308856238529041437143701491906725139139273618492636956911128555920390837527157952466944428573627214533749990909264153400965193482282064240548108941615002635738792420617452437682964218211751328042659835850657769374386579370568470549750371038709396183130319610808591337460135526416948491932549810737199655045455331391450382983019002641609727028567508155846045247898180626508526373744608336527126456893489195593967111

## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [47]:
import random, sys, os

def generateKey(keySize):
    #napisz swój kod tutaj

    p = generatePrime(keySize)
    q = generatePrime(keySize)

    n = p * q

    e_max = (p-1) * (q-1)

    e = random.randrange(3, e_max)
    while gcd(e, e_max) != 1 :
        e = random.randrange(3, e_max)

    d = findModInverse(e, e_max)

    publicKey = n, e
    privateKey = n, d

    return publicKey, privateKey

  
print('Generujemy klucze publiczny i prywatny')
public_key, private_key = generateKey(512)
print(public_key, "\n", private_key)

Generujemy klucze publiczny i prywatny
(136910366345544030943138397179367108492400131190345692867838528754668862894224654505150155252279375577470043936792900192531796642551183956823592737402547078045221908173157536511084789958877859591220751049226326165839033909185411251109962982294312134400567621938083486894523754566916405332490005700294765799827, 104383719163095207868761096685601363108363808782602242621801026386737916896376752556933347780757663163141108053946432494968146628366313842098692667144111219242264288738182602418497957311278100590520137094717139401259065426297011812270600390100280750629070916294413254338359355611368640702937786673593651304533) 
 (136910366345544030943138397179367108492400131190345692867838528754668862894224654505150155252279375577470043936792900192531796642551183956823592737402547078045221908173157536511084789958877859591220751049226326165839033909185411251109962982294312134400567621938083486894523754566916405332490005700294765799827, 361220600669790421619318

## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [48]:
def encrypt(message, modulus, exp):
    message_encrypted = [None] * len(message)

    for i, c in enumerate(message):
        message_encrypted[i] = pow(ord(c), exp, modulus)

    # kod szyfrowania     
  
    return message_encrypted


message = "hi rsa"

encrypted = encrypt(message, public_key[0], public_key[1])

print(encrypted)

[6718557481577128092235637847829717860252912883074623909676032646095877306082094893500656233026522725260109628857945677803373713819576616279597438107315914680354701785779884708954589760128146720848027776447340591977559698246395577800021629846318043567469675250854404964641085327624023736916361222319914192448, 11751842390038117266673526446647264980174724460196073296975423187917872801785637318930260337078058409134143715717762621570183571073827471388010229186911019015469290812888022778554312233320204071088898718952727814440288385797917752311127349311637529517207853800763439969747373453586469811329042353584661966910, 80891742555387119693060758174366594168802521313803491805375541357414522635200514391252688827199738968975394414121395928601468352466339060308879796727701642160543653428641456609384005061512426365265847870333570714993740651143593520682022776357286778330440778412917628177294675028987977391029564176470498626279, 1115697067264707919976623392037739613750192856132636961036746981155338

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [49]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = [None] * len(message_encrypted)

    for i, a in enumerate(message_encrypted):
        message_ascii[i] = chr(pow(a, exp, modulus))

    return ('').join(message_ascii)

decrypted = decrypt(encrypted, private_key[0], private_key[1])

print(decrypted)

hi rsa


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie 
Zademonstruj deformowalność podręcznikowego szyfrowania RSA

oaep na aktywnosc